<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Практическая работа № 11. Стратегии трансферного обучения для задач обработки естественного языка**

## **1. Цель и задачи работы**

Цель работы — формирование у обучающегося системного представления о конвейере трансферного обучения в задачах NLP, освоение практических навыков проектирования, реализации и сравнительного анализа стратегий адаптации предобученных моделей под целевые задачи с различным объёмом данных и архитектурными требованиями, а также приобретение компетенций в области анализа влияния исходного домена, калибровки предсказаний, устойчивости к лингвистическим возмущениям и обеспечения воспроизводимости результатов в соответствии с современными стандартами открытой науки.

**Основные задачи работы:**  
1. Использовать корпус текстов, сформированный в рамках Практической работы № 1 и размеченный в Практической работе № 3, в качестве экспериментальной базы.  
2. Выполнить адаптацию моделей к трём разнородным задачам:  
 – **бинарная классификация** (тональность, ≤1 000 примеров),  
 – **последовательная разметка** (NER: PER, ORG, LOC, ~5 000 токенов),  
 – **sequence-to-sequence генерация** (абстрактное суммаризирование, ~3 000 пар).  
3. Реализовать программные модули для пяти стратегий трансферного обучения:  
 – *feature extraction* (заморозка энкодера),  
 – *full fine-tuning* (дообучение всей модели),  
 – *layer-wise learning rates* (дифференцированное обучение слоёв),  
 – *adapter-based tuning*,  
 – *LoRA (Low-Rank Adaptation)*.  
4. Провести сравнительный анализ влияния стратегии адаптации, типа предобученной модели (общая, специализированная, мультиязычная) и размера целевого набора на качество, скорость и устойчивость.  
5. Освоить практики строгого обучения: мониторинг сходимости, ранняя остановка, регуляризация (dropout, weight decay, label smoothing, mixout), визуализация кривых обучения.  
6. Применить методы оценки устойчивости: paraphrasing, back-translation, и калибровки: reliability diagrams, Expected Calibration Error (ECE), isotonic regression.  
7. Проанализировать изменения в внутренних представлениях модели: проекции эмбеддингов (UMAP/t-SNE), attention-карты до/после адаптации, linear probing по слоям.  
8. Разработать унифицированный и конфигурируемый пайплайн адаптации, поддерживающий смену задачи, модели и стратегии без модификации кода.  
9. Обеспечить воспроизводимость и открытость: опубликовать размеченные поднаборы, программный код, все fine-tuned модели и аналитический отчёт, включая развёртывание демо-приложения на Hugging Face Spaces.

---

## **2. Теоретические предпосылки**

Трансферное обучение в NLP эволюционировало от статических эмбеддингов к динамическим, контекстуализированным представлениям, предоставляемым трансформерными архитектурами. Современные предобученные модели (BERT, RoBERTa, T5, BART) служат универсальными «языковыми приорами», которые могут быть эффективно адаптированы к специфическим задачам даже при ограниченных данных.

В зависимости от стратегии адаптации выделяют три ключевых направления:  
1. **Полный трансфер (Full Fine-Tuning):**  
 – Все параметры модели обновляются;  
 – Обеспечивает наивысшее качество, но требует значительных вычислительных ресурсов и рискует переобучением на малых данных.  
2. **Параметрически эффективная адаптация (Parameter-Efficient Transfer Learning, PEFT):**  
 – **Adapters**: вставка небольших bottleneck-слоёв между подблоками трансформера;  
 – **LoRA**: аппроксимация обновлений весов внимания через низкоранговые матрицы;  
 – Позволяют обучать <1% параметров при сохранении >95% качества full fine-tuning.  
3. **Фиксированное извлечение признаков (Feature Extraction):**  
 – Энкодер заморожен, обучается только голова;  
 – Быстрый и устойчивый подход для ultra-low-resource сценариев, но менее гибкий.

Качество трансфера напрямую зависит от:  
– **семантической близости** домена предобучения и целевой задачи,  
– **совместимости архитектуры** (encoder-only vs encoder-decoder),  
– **баланса между обобщением и специализацией** (регуляризация, learning rate schedule),  
– **калибровки вероятностей**, особенно критичной для задач с дисбалансом или safety-critical применения.

Анализ внутренних представлений (через probing, attention visualisation, embedding projections) позволяет понять, **что модель сохраняет, а что теряет** при адаптации — ключевой аспект для интерпретируемости и доверия.

---

## **3. Порядок выполнения работы**

### **3.1. Формирование размеченного экспериментального корпуса**

**Задача:** Подготовить три поднабора на основе данных из Практической работы № 1.  
**Требования к выполнению:**  
– Источники: ТАСС, РИА Новости, Lenta.ru, Meduza, Коммерсант, belgech.ru;  
– Типы задач и разметки:  
 ✓ **Sentiment-1K**: бинарная тональность («позитив»/«негатив»), ≤1 000 документов;  
 ✓ **RuNER-5K**: разметка сущностей (PER, ORG, LOC) в формате CoNLL, ≥5 000 токенов;  
 ✓ **NewsSumm-3K**: пары (заголовок, текст) для абстрактного суммаризирования, ≥3 000 пар;  
– Минимальный объём: как указано выше;  
– Структура: JSONL с полями `text`, `sentiment`, `ner_tags`, `summary`;  
– Разделение: train/validation/test в соотношении 70/15/15 с сохранением стратификации (для классификации и NER) или случайным разбиением (для summarization);  
– Гарантия отсутствия пересечений между поднаборами.

### **3.2. Формирование признаковых представлений и выбор моделей-доноров**

**Задача:** Создать входные представления и выбрать предобученные модели.  
**Требования:**  
– Модели-доноры:  
 ✓ `DeepPavlov/rubert-base-cased` (общий русский BERT),  
 ✓ `sberbank-ai/ruRoberta-large` (усовершенствованная RoBERTa),  
 ✓ `cointegrated/rubert-tiny2` (компактная модель),  
 ✓ `microsoft/mdeberta-v3-base` (мультиязычная DeBERTa),  
 ✓ `IlyaGusev/ruRoBERTa-finance` (специализированная, при наличии),  
 ✓ `google/mt5-base` (для summarization);  
– Для классификации и NER: использование `AutoTokenizer.from_pretrained`;  
– Для summarization: encoder-decoder токенизация;  
– Формирование тензоров: `input_ids`, `attention_mask`, `labels`;  
– Сохранение метаданных о конфигурации токенизатора в JSON.

### **3.3. Реализация модуля стратегий трансферного обучения**

**Задача:** Разработать конфигурируемый модуль для адаптации моделей.  
**Функциональные требования:**  
– Поддержка:  
 ✓ Feature extraction (заморозка backbone),  
 ✓ Full fine-tuning (с общим и дифференцированным LR),  
 ✓ Adapter tuning (Houlsby-style),  
 ✓ LoRA (для attn-проекций),  
 ✓ Linear probing (для анализа);  
– Единый интерфейс `TransferAdaptor`;  
– Поддержка задач: классификация, токен-классификация (NER), seq2seq;  
– Поддержка GPU через PyTorch + Accelerate;  
– Код в модуле `transfer_adaptors.py` с документированием и типизацией.

### **3.4. Обучение моделей и стратифицированная оценка**

**Задача:** Провести строгое обучение с мониторингом.  
**Требования:**  
– Использование отдельной валидационной выборки (15%) для early stopping;  
– Метрики:  
 ✓ F1 (macro) — для классификации,  
 ✓ F1 (entity-level) — для NER,  
 ✓ ROUGE-L — для summarization;  
– Визуализация:  
 ✓ Графики loss и метрик по эпохам (train/val),  
 ✓ Learning curves при разных размерах данных,  
 ✓ Нормы градиентов по слоям (для layer-wise LR);  
– Сохранение: конфигураций, весов, логов в JSON и PyTorch.

### **3.5. Сравнительный анализ стратегий трансфера**

**Задача:** Эмпирически оценить эффективность подходов.  
**Метрики:**  
– Качество: F1, ROUGE, accuracy;  
– Эффективность: время обучения, объём обновляемых параметров, пиковое потребление GPU RAM;  
– Устойчивость: дисперсия метрик по случайным сидам.  
**Форма представления:** таблица `transfer_comparison.csv`, boxplots, radar-диаграммы «качество–ресурс–скорость».

### **3.6. Анализ влияния размера целевого набора**

**Задача:** Исследовать зависимость качества от объёма данных.  
**Требования:**  
– Для каждой задачи: подвыборки размером 100, 300, 1 000, 3 000, 10 000;  
– Обучение RuBERT всеми стратегиями на каждой подвыборке;  
– Построение **learning curves** с доверительными интервалами;  
– Выявление «точек насыщения» — где добавление данных перестаёт давать выигрыш.

### **3.7. Оценка устойчивости и калибровки**

**Задача:** Обеспечить надёжность и достоверность предсказаний.  
**Функциональные требования:**  
– **Парфраз-атака**: генерация 3–5 переформулировок через T5-paraphraser, оценка стабильности предсказаний;  
– **Калибровка**:  
 ✓ Построение reliability diagrams,  
 ✓ Расчёт ECE,  
 ✓ Применение Platt scaling и isotonic regression;  
– Оценка по метрикам: Brier score, NLL.

### **3.8. Интерпретация процесса трансфера**

**Задача:** Исследовать, как меняются внутренние представления.  
**Функциональные требования:**  
– **Linear probing**: обучение линейного классификатора на выходах каждого слоя до/после адаптации;  
– **Проекции**: UMAP/t-SNE на [CLS]-эмбеддингах и токенных представлениях;  
– **Визуализация внимания**: сравнение attention-карт в RuBERT до и после fine-tuning (bertviz);  
– Анализ «забывания»: падение качества на исходной задаче предобучения (если применимо).

### **3.9. Публикация и развёртывание результатов**

**Задача:** Обеспечить открытость и доступность.  
**Требования:**  
– Публикация кода на GitHub/GitLab под лицензией MIT;  
– Публикация поднаборов на Hugging Face Datasets;  
– Публикация всех fine-tuned моделей на Hugging Face Hub с **Model Card**, включающим:  
 ✓ Описание задачи и стратегии,  
 ✓ Гиперпараметры, метрики, объём данных,  
 ✓ Пример кода инференса,  
 ✓ Лицензию и ограничения;  
– Развёртывание демо-панели на Hugging Face Spaces (Streamlit/Gradio) с возможностью:  
 ✓ Выбора задачи, модели и стратегии,  
 ✓ Ввода текста,  
 ✓ Отображения предсказания, вероятностей, attention-карты,  
 ✓ Сравнения нескольких моделей на одном примере.

---

## **4. Дополнительные исследовательские задания**

1. **Сравнение PEFT-методов**: Adapters vs LoRA — качество, скорость, потребление памяти.  
2. **Влияние домена предобучения**: насколько финансы-модель проигрывает общему RuBERT на общей задаче?  
3. **Zero-shot через prompting**: сравните zero-shot (через шаблоны) и few-shot (100 примеров) на тональности.  
4. **Экономика трансфера**: измерьте CO₂ и стоимость обучения через CodeCarbon.  
5. **Multi-task адаптация**: обучите одну модель на тональности и NER — есть ли положительный перенос?  
6. **Калибровка после PEFT**: ухудшается ли калибровка при использовании LoRA?  
7. **Перенос между языками**: применим ли mDeBERTa к русскому NER лучше, чем RuBERT?  
8. **Анализ слоёв через probing**: какие слои несут синтаксическую, а какие — семантическую информацию после адаптации?

---

## **5. Требования к отчёту**

Отчёт оформляется в соответствии с **ГОСТ 7.32–2017** и должен содержать:  
1. **Введение** — постановка задачи, актуальность трансферного обучения, обзор эволюции стратегий (от feature extraction к PEFT);  
2. **Методология** — описание пайплайнов, моделей, стратегий, метрик, подходов к калибровке и устойчивости;  
3. **Результаты** — таблицы, графики кривых обучения, learning curves, reliability diagrams, UMAP-проекции, attention-карты до/после;  
4. **Обсуждение** — интерпретация компромиссов «качество–ресурс–обобщение», анализ «точек разрыва» трансфера, сравнение с классическими методами (Работа № 4);  
5. **Заключение** — выводы и рекомендации по выбору стратегии трансфера для low-resource, multilingual и safety-critical сценариев;  
6. **Список источников** — по ГОСТ Р 7.0.5–2008;  
7. **Приложения** — фрагменты кода, скриншоты демо-панели, примеры разметки, конфигурации адаптации.

Отчёт сопровождается ссылками на:  
– Репозиторий с кодом;  
– Hugging Face Dataset с поднаборами;  
– Hugging Face Space с веб-демо;  
– Опубликованные fine-tuned модели с Model Card.

---

## **6. Критерии оценивания**

| Оценка | Критерии |
|--------|---------|
| **Отлично** | Реализованы ≥4 стратегии на ≥2 задачах; анализ влияния размера данных и домена предобучения; калибровка и устойчивость; интерпретация через probing и attention; функциональный веб-интерфейс на Hugging Face Spaces; публикация всех моделей с Model Card; отчёт по ГОСТ с глубоким обсуждением компромиссов и рекомендациями. |
| **Хорошо** | Реализованы 2–3 стратегии (full tuning, adapters, LoRA); обучение на всех трёх задачах; базовый анализ устойчивости и калибровки; корректный отчёт с визуализациями. |
| **Удовлетворительно** | Реализованы feature extraction и full fine-tuning на одной задаче; отчёт содержит описание методов и таблицы метрик. |
| **Неудовлетворительно** | Отсутствует сравнение стратегий; нет анализа влияния размера данных или интерпретации; не опубликованы модели или код. |
